In [1]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from datetime import datetime
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By


In [2]:
driver = webdriver.Chrome()
driver.get("https://www.europeanlemansseries.com/season")

# Wait for dynamic content to load if needed
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "season-calendar__list-item")))


sched_soup = BeautifulSoup(driver.page_source, 'html.parser')
schedule_details = sched_soup.find_all('li', {'class':"season-calendar__list-item"})

driver.quit()

# <li class="season-calendar__list-item"><a href="/race/4805" class="link event-link -future" target="_self"><!----><img src="https://www.fiawec.com/bundles/easycore/img/flag/es.png" alt="France" class="event-link__flag flag"><div><span class="event-link__name">Official Tests</span><br><time class="event-link__date">09 April 2024</time></div></a></li>


In [3]:
# schedule_details

In [4]:
days = [] 
months = []
titles = [] 
descs = []
circuits = []
time = []
start_hours = []
start_minutes = []
end_hours = []
end_minutes = []

year = datetime.now().year

round_counter = 1


month_mapping = {
        'Jan': 1,
        'JAN': 1,
        'January': 1,
        'Feb': 2,
        'FEB': 2,
        'February': 2,
        'Mar': 3,
        'March': 3,
        'Apr': 4,
        'April': 4,
        'May': 5,
        'Jun': 6,
        'June': 6,
        'Jul': 7,
        'July': 7,
        'Aug': 8,
        'August': 8,
        'Sep': 9,
        'September': 9,
        'Oct': 10,
        'October': 10,
        'Nov': 11,
        'November': 11,
        'Dec': 12,
        'DEC': 12,
        'December': 12
    }



In [5]:
##Example building info lists

# for i in schedule_details:
#     temp_circuit = i.find('div', {'class': 'race-circuit'}).text
#     circuits.append(temp_circuit)
    
#     temp_date = i.find('div', {'class':"race-date"}).text
#     temp_date = temp_date.split(' ')
#     days.append(int(temp_date[0]))
    
#     temp_month = temp_date[1]
#     converted_month = month_mapping[temp_month]
#     months.append(converted_month)
    
#     round_counter = 1
#     temp_title = 'AsLMS Round' + ' ' + str(round_counter)
#     round_counter += 1
#     titles.append(temp_title)

In [6]:
# <div><span class="event-link__name">Official Tests</span><br/><time class="event-link__date">09 April 2024</time></div>

In [7]:

for i in schedule_details:
    temp_date = i.find('time').text.split()
    print(temp_date)

['09', 'April', '2024']
['14', 'April', '2024']
['05', 'May', '2024']
['07', 'July', '2024']
['25', 'August', '2024']
['29', 'September', '2024']
['19', 'October', '2024']


In [8]:


for i in schedule_details:
    temp_date = i.find('time').text.split()
    
    day = int(temp_date[0])
    temp_month = temp_date[1]
    month = month_mapping[temp_month]
    days.append(day)
    months.append(month)
    
    desc = i.find('span').text
    descs.append(desc)
    
    if desc == 'Official Tests':
        titles.append('ELMS Official Tests')
        
    else:
        temp_title = 'ELMS Round' + ' ' + str(round_counter)
        round_counter += 1
        titles.append(temp_title)

    start_hour = 11
    start_hours.append(start_hour)
        
    end_hour = start_hour + 4
    end_hours.append(end_hour)

    

In [9]:
descs

['Official Tests',
 'Barcelona',
 'Le Castellet',
 'Imola',
 'Spa-Francorchamps',
 'Mugello',
 'Portimão']

In [10]:
start_hours

[11, 11, 11, 11, 11, 11, 11]

In [11]:
end_hours

[15, 15, 15, 15, 15, 15, 15]

In [12]:
titles

['ELMS Official Tests',
 'ELMS Round 1',
 'ELMS Round 2',
 'ELMS Round 3',
 'ELMS Round 4',
 'ELMS Round 5',
 'ELMS Round 6']

In [13]:
import pytz
import icalendar
from datetime import datetime
from pathlib import Path
import os
from icalendar import Calendar, Event,  vText, vCalAddress

In [14]:
ELMS = Calendar()
ELMS.add('prodid', '-//My calendar product//example.com//')
ELMS.add('version', '2.0')
uid = 110

In [15]:
 
for title, desc, day, month, s_h, e_h in zip(titles, descs, days, months, start_hours, end_hours):
    ievent = Event()
    ievent.add('summary', title) #Title of the event
    ievent.add('description', desc)     #Description of event
    #     ievent.add('description', f"{event}\n{length}")

    ievent.add('dtstart', datetime(year, month, day, s_h, 0, 0, tzinfo=pytz.timezone('Europe/Paris')))
    ievent.add('dtend', datetime(year, month, day, e_h, 0, 0, tzinfo=pytz.timezone('Europe/Paris')))
    ievent.add('dtstamp', datetime(year, 1, 16, 0, 10, 0, tzinfo=pytz.timezone('est')))
#     ievent.add('location', circuit)
    ievent.add('uid', uid)
    uid += 1
    ELMS.add_component(ievent)

In [16]:
ELMS

VCALENDAR({'PRODID': vText('b'-//My calendar product//example.com//''), 'VERSION': vText('b'2.0'')}, VEVENT({'SUMMARY': vText('b'ELMS Official Tests''), 'DESCRIPTION': vText('b'Official Tests''), 'DTSTART': <icalendar.prop.vDDDTypes object at 0x0000017D1A6B0BE0>, 'DTEND': <icalendar.prop.vDDDTypes object at 0x0000017D1A6B0C70>, 'DTSTAMP': <icalendar.prop.vDDDTypes object at 0x0000017D1A6B0D90>, 'UID': vText('b'110'')}), VEVENT({'SUMMARY': vText('b'ELMS Round 1''), 'DESCRIPTION': vText('b'Barcelona''), 'DTSTART': <icalendar.prop.vDDDTypes object at 0x0000017D1A6B0EE0>, 'DTEND': <icalendar.prop.vDDDTypes object at 0x0000017D1A6B0F70>, 'DTSTAMP': <icalendar.prop.vDDDTypes object at 0x0000017D1A723040>, 'UID': vText('b'111'')}), VEVENT({'SUMMARY': vText('b'ELMS Round 2''), 'DESCRIPTION': vText('b'Le Castellet''), 'DTSTART': <icalendar.prop.vDDDTypes object at 0x0000017D1A723190>, 'DTEND': <icalendar.prop.vDDDTypes object at 0x0000017D1A723220>, 'DTSTAMP': <icalendar.prop.vDDDTypes object a

In [17]:
directory = str(Path(r'C:\Users\chris\Documents\Calendars')) + "/"
print("ics file will be generated at ", directory)
f = open(os.path.join(directory, 'ELMS.ics'), 'wb')
f.write(ELMS.to_ical())
f.close()

ics file will be generated at  C:\Users\chris\Documents\Calendars/
